In [3]:
import pandas as pd
import numpy as np
from numpy import array

edgar_dataset = '/home/cj/Bureau/Master2/bdp/database/edgar_database_2018.xls'

# Data sheets of the emissions of CO2 by sectors depending on the country
co2_sector_country = 'fossil_CO2_by_sector_and_countr'
data_co2_sector_country = pd.read_excel(open(edgar_dataset, 'rb'),
                                  sheet_name = co2_sector_country)

# 1 : Data set #

## We consider Belgium CO2 emissions by sectors. ##

In [4]:
data_co2_sector_country.head()

,Sector,country_name,1970,1971,1972,1973,1974,1975,1976,1977,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Power Industry,Afghanistan,0.056962,0.056962,0.116895,0.174969,0.206875,0.210758,0.241009,0.375716,...,1.588591,2.017081,2.116617,2.100720,3.233659,3.414392,3.241768,3.241768,3.322330,3.419763
1,Power Industry,Albania,0.600624,0.600624,0.616104,0.613008,0.606816,0.532512,0.535608,0.510840,...,0.006192,0.015480,0.030960,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Power Industry,Algeria,1.645869,1.645869,1.550235,1.221972,1.465168,2.111895,2.530895,2.819267,...,24.666333,25.086252,28.120880,30.929377,30.094453,32.629870,36.774115,36.177305,36.416426,39.915370
3,Power Industry,Angola,0.137546,0.137546,0.159283,0.168752,0.371864,0.352957,0.346584,0.346584,...,2.226396,2.369069,2.229655,1.843470,2.325888,3.441893,3.775559,3.974273,3.967023,4.066821
4,Power Industry,Anguilla,0.000430,0.000430,0.000436,0.000438,0.000447,0.000480,0.000521,0.000449,...,0.000778,0.000790,0.000910,0.000910,0.000978,0.000976,0.000980,0.000990,0.000985,0.001011


In [88]:
dataset = data_co2_sector_country.loc[data_co2_sector_country['country_name'] == 'Belgium']
sector_names = dataset['Sector'].to_list()
del dataset['country_name']
del dataset['Sector']

dataset = dataset.transpose()
dataset.columns = sector_names

# A column representing the sum of all sectors is added to the dataset.
Total = np.zeros(len(dataset))
Total = pd.Series(Total)
Total = dataset.iloc[:,1] + dataset.iloc[:,2]
for i in range(5) : Total = Total + dataset.iloc[:,i]
    
dataset['Total'] = Total

dataset = dataset.astype('float32')
dataset.head()

,Power Industry,Buildings,Transport,Other industrial combustion,Other sectors,Total
1970,27.047808,35.884159,11.413301,50.741562,15.272869,187.657166
1971,29.284933,33.066238,11.830670,43.790886,12.166234,175.035873
1972,32.564274,36.309933,12.646909,46.441601,13.717597,190.637161
1973,35.604969,36.831837,12.949210,47.686749,13.476537,196.330338
1974,38.689392,33.134407,12.561368,46.169109,14.476633,190.726685


In [10]:
# save dataset
dataset.to_csv('belgium_consumption_sector.csv')

# 2. Problem framing #
Given the consumption of CO2 of different sectors, what is the expected annual CO2 consumption fot the 12 next years ?
This requires that a predictive model forcasts the total active power for each year over the next 12 years. This is called *multivariate multi-step time series Forcasting model.*

Why 12 years ? Because this dataset stops in 2018 and we are asked to predict the emissions of CO2 in 2030.

# 3. Resolution #

In [11]:
from pandas import read_csv
dataset = read_csv('/home/cj/Documents/belgium_consumption_sector.csv',
                   infer_datetime_format=True, index_col=['Unnamed: 0'])

## Split the dataset ## 

In [12]:
def split_dataset(X, m): # m is the size of the validation set & X the dataset.
    n_train = len(X) - m
    train = X[0:n_train]
    test = X[n_train:len(X)]
    return train, test

## Convert the multivariate matrix into an univariate matrix  ##

In [28]:
# Convert the multivariate matrix into an univariate matrix.
# We want to forecast one row of the dataset.
def to_series(data, var):
    # extract the wanted variable
    series = [row[var] for row in data]
    # flatten into a single series
    series = array(series).flatten()
    return series

## Pipeline ##

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

def make_pipeline(model):
    steps = list()
    # standardization
    steps.append(('standardize', StandardScaler()))
    # normalization
    steps.append(('normalize', MinMaxScaler()))
    # the model
    steps.append(('model', model))
    # create pipeline
    pipeline = Pipeline(steps=steps)
    return pipeline

## One-step forecast in done ##

In [30]:
def forecast(model, input_x, n_input):
    yhat_sequence = list()
    input_data = [x for x in input_x]
    
    # prepare the input data
    X = array(input_data[-n_input:]).reshape(1, n_input)
    # make a ONE-STEP forecast
    yhat = model.predict(X)[0]
    # add to the result
    yhat_sequence.append(yhat)
    # add the prediction to the input
    input_data.append(yhat)
        
    return yhat_sequence

## Creation of the supervided dataset ##

In [69]:
# converts the window into train and test sets
def train_test_datasets(window, n_input, var):
    
    # convert the window of data into an univariate serie
    data = to_series(window, var) 
    
    X, y = list(), list()
    
    ix_start = 0
    # step over the entire window one time step at a time
    
    for i in range(len(data)):
        # define the end of the input sequence
        ix_end = ix_start + n_input
        # ensure we have enough data for this instance
        if ix_end < len(data):
            X.append(data[ix_start:ix_end])
            y.append(data[ix_end])
        # move along one time step
        ix_start += 1
    
    train_x_y = []; train_x_y.append(array(X)); train_x_y.append(array(y))
    return train_x_y

## Train then forecast ##

In [79]:
def model_prediction(model, window, n_input, var):
    # prepare data
    train_x_y = train_test_datasets(window, n_input, var)
    train_x = train_x_y[0]
    train_y = train_x_y[1]
    # make pipeline
    pipeline = make_pipeline(model)
    # fit the model
    pipeline.fit(train_x, train_y)
    
    last_year = train_y[len(train_y)-1] # last value of the training set
    # this value is added to the 4 values of train_x in order to form the input
    input_row = np.append(train_x[len(train_x)-1, 1:n_input], last_year)   
    
    # predict the next year knowing the n_input last years
    yhat_sequence = forecast(pipeline, input_row, n_input)
    return yhat_sequence

## Evaluation of each model ##

In [80]:
def evaluate_model(model, train, test, n_input, var): # FOR ONE VARIABLE AT THE TIME.
    # window is a list of weekly data
    window = [x for x in train]
    # walk-forward validation over each week
    predictions = list()
    for i in range(len(test)): # 12 times, you predict 1 by 1
        # predict the week
        yhat_sequence = model_prediction(model, window, n_input, var)
        # store the predictions
        predictions.append(yhat_sequence)
        # get real observation and add to window for predicting the next week
        window.append(test[i, :]) # no need to give all the lines unless you predict all the variable at the same time which will be done
    predictions = array(predictions)
   
    return predictions

## How a model performed ? ##

In [81]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

def evalutation_forecast(real, pred):
    pred = to_pd_series(pred)
    real = to_pd_series(real)
    #diff = list()
    #for i in range(len(real)): diff.append(abs(real[i]-pred[i]))
    
    #mse = mean_squared_error(real, pred)
    #rmse = sqrt(mse)
    mae = mean_absolute_error(real, pred)
    
  
    return mae

## Plot the forcasting ##

In [82]:
def to_pd_series(data):
    data = [x for x in data]
    data = array(data).flatten()
    data = pd.Series(data)
    
    return data

In [83]:
def model_plot(fig, pred, name):
    fig.add_trace(go.Scatter(
                x=years,
                y=to_pd_series(pred),
                name=name,
                opacity=0.8))
    return fig

In [84]:
def sort_models(eval_model):
    models_sorted = sorted(eval_model.items(), key=lambda x: x[1], reverse=False)
    return models_sorted
    

In [85]:
def print_evaluation(sector, models_sorted):
    print(sector)
    for i in range(len(models_sorted)) :
        print('{}. {} : {}'.format(i+1, models_sorted[i][0], models_sorted[i][1]))

## Linear models tested ##

In [86]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import Lars
from sklearn.linear_model import LassoLars
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import SGDRegressor

models = dict()
# LINEARS MODELS ONLY
models['lr'] = LinearRegression()
models['lasso'] = Lasso()
models['ridge'] = Ridge()
models['sgd'] = SGDRegressor(max_iter=5000, tol=1e-3)
models['pa'] = PassiveAggressiveRegressor(max_iter=1000, tol=1e-3)
models['en'] = ElasticNet()
models['huber'] = HuberRegressor()
models['lars'] = Lars()
models['llars'] = LassoLars()
models['ranscac'] = RANSACRegressor()

## Main ##

In [87]:
from matplotlib import pyplot
import plotly.graph_objects as go

train, test = split_dataset(dataset.values, 12)

n_input = 6

years = pd.Series(dataset.index.values[-12:])

variables = [0, 1, 2, 3, 4, 5]
figures = list()
for i in range(len(variables)): figures.append(go.Figure())
list_eval_model = list()

for var in variables:
    eval_model = dict()
    for name, model in models.items():
    
        pred = evaluate_model(model, train, test, n_input, var)
        mae = evalutation_forecast(test[:,var], pred)
    
        eval_model[name] = mae
    
        figures[var] = model_plot(figures[var], pred, name)
    
    list_eval_model.append(eval_model)
    



## Best models evaluated with MAE for each sector ##

In [67]:
name_sector = {0:'Power industry', 1:'Buildings', 2:'Transport',
               3:'Other industrial combustion', 4:'Other sectors', 5:'Total'}

for i, sector in name_sector.items():
    print_evaluation(sector, sort_models(list_eval_model[i]))
    print('\n')

Power industry
1. ranscac : 1.1678842572082673
2. lr : 1.3587070924331703
3. lars : 1.358707092433171
4. sgd : 1.3717103709000955
5. huber : 1.4446913708339313
6. pa : 1.70580901181051
7. ridge : 2.0623567815684303
8. en : 6.197874605577567
9. lasso : 6.53938799119253
10. llars : 6.618436754200076


Buildings
1. sgd : 2.0539296754781016
2. ridge : 2.406808962700204
3. lars : 2.5165886793370054
4. lr : 2.5344556634171838
5. pa : 2.5809781504259637
6. huber : 2.6428260626005367
7. ranscac : 4.158651021165017
8. en : 4.394498819073153
9. lasso : 4.400564330633393
10. llars : 4.400564330633393


Transport
1. ranscac : 0.6547496123297298
2. huber : 0.6589214438724452
3. lr : 0.6639135294668236
4. lars : 0.6754837817638689
5. ridge : 1.0117243713603206
6. sgd : 1.2873979396406083
7. pa : 1.435150059771886
8. en : 2.7458495836805725
9. lasso : 3.3576564835848153
10. llars : 4.508266031729942


Other industrial combustion
1. pa : 0.8470033747222713
2. lars : 1.1418621251076464
3. lr : 1.141862

Comments : 'Other sectors' is very badly predicted using linear predictors. We need to compare total CO2 emissions
with and without this particular sector. The total (sum of all columns is totally wrong !). You can also take the sum of all sectors (with their best predictor).

# PLOT #

In [68]:
years = pd.Series(dataset.index.values[-12:])

for var in variables:

    figures[var].add_trace(go.Scatter(
                    x=years,
                    y=to_pd_series(test[:,var]),
                    name='Real',
                    line=go.scatter.Line(color="black"),
                    opacity=0.1))

    figures[var].update_layout(title_text="Carbon dyoxide emissions by "+ name_sector[var],
                      xaxis={"title":"Years"}, yaxis={"title":"Carbone dioxyde (Mt/yr)"})
    figures[var].show()
    

Except for the Power industry, all other sectors are badly predicted as aswell as 'Total'.

# 6. FROM 2019 TO 2030 #

### TODO ###
- comments
- read comments
- read on linear models 
- Test the statinnarity of all variables
- sliding window (done)
- forward chaining 
- downsampling
- LTSM 
- Autoregressive model
- from 2019 - 2030 + graphs
- transformée de fourier rapide 
- pourquoi sont-elles mals prédites ?
- pourquoi il ya de fortes variations dans les données réelles ?
- improve hyperparamter 
- conditions for using models ? Maybe 
- model evaluation => pay attention to interpretation